https://www.kaggle.com/datasets/shivamb/machine-predictive-maintenance-classification/data

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

60000 PIEZAS AL MES

SON 3000 AL DIA
CON UN COSTO 1500 PERO SI ES NECESARIO TERCIARIZAR PASAR A SER DE 4500 - Al Costo planificado se le suman 3000 pesos
$9.000.000 en perdidas en el dia en caso de parar

Son 10000 registros , suponiendo que fueron en el año

Cada paro requiere 1 dia (9 horas) de trabajo Perdida 9.000.000

Cada preventivo requiere 1 hora de trabajo Ganancia 8.000.000

Segun el dataset hubo 343 paros, representa una perdida de 3.087.000.000 anual

NO CONSIDERO GASTOS EN REPUESTOS Y MANO DE OBRA!!! A fin de simplificar la cuenta

In [2]:
#Funcion de ganancia
def asignar_puntajes(real, predicho):
    if real == 0 and predicho == 1: #APLICAS UN MANTENIMIENTO INNECESARIO
        return -1000000
    elif real == 1 and predicho == 1: #APLICAS UN MANTENIMIENTO CORRECTO
        return 8000000
    elif real == 0 and predicho == 0: #NO SE HACE NADA
        return 0
    elif real == 1 and predicho == 0: #ERA NECESARIO UN MANTENIMIENTO
        return -9000000
    else:
        return 0

In [3]:
df = pd.read_csv('predictive_maintenance.csv')
df.shape

(10000, 10)

**FEATURE ENGINEERING**

In [4]:
df.head(3)

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure


In [5]:
df = pd.DataFrame(df)

In [6]:
df = df.drop(['Product ID'], axis=1)
df = df.drop(['UDI'], axis = 1)
#type_dummie = pd.get_dummies(df['Type'])
#df2 = pd.concat([df, type_dummie], axis=1)    
#df2.drop(['Type'], inplace=True, axis=1)
#Failure_dummie = pd.get_dummies(df['Failure Type'])
#df3 = pd.concat([df2, Failure_dummie], axis=1)    
df = df.drop(['Failure Type'], axis = 1)
df = df.drop(['Type'], axis = 1)

**FeatureTools**

In [7]:
import featuretools as ft

C:\ProgramData\anaconda3\Lib\site-packages\dask\dataframe\_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 13.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [8]:
es = ft.EntitySet("maintenance")

es = es.add_dataframe(
    dataframe_name="maintenance",
    dataframe=df,
    index="UDI"#,
    #time_index="YrSold"#,
    #logical_types={
     #   "product_id": Categorical,
      #  "zip_code": PostalCode,
    #},
)

C:\Users\djoglar\AppData\Roaming\Python\Python311\site-packages\featuretools\entityset\entityset.py:1914: UserWarning: index UDI not found in dataframe, creating new integer column
  warnings.warn(


In [9]:
es

Entityset: maintenance
  DataFrames:
    maintenance [Rows: 10000, Columns: 7]
  Relationships:
    No relationships

In [10]:
feature_matrix, feature_defs = ft.dfs(entityset=es,
                                      target_dataframe_name="maintenance",
                                      max_depth=1)

In [11]:
feature_defs

[<Feature: Air temperature [K]>,
 <Feature: Process temperature [K]>,
 <Feature: Rotational speed [rpm]>,
 <Feature: Torque [Nm]>,
 <Feature: Tool wear [min]>,
 <Feature: Target>]

In [12]:
feature_matrix

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target
UDI,,,,,,
0,298.1,308.6,1551,42.8,0,0
1,298.2,308.7,1408,46.3,3,0
2,298.1,308.5,1498,49.4,5,0
3,298.2,308.6,1433,39.5,7,0
4,298.2,308.7,1408,40.0,9,0
...,...,...,...,...,...,...
9995,298.8,308.4,1604,29.5,14,0
9996,298.9,308.4,1632,31.8,17,0
9997,299.0,308.6,1645,33.4,22,0


In [13]:
#ft.list_primitives()

No crea ningun campo nuevo!

problemas encontrados con featureTools:

-Al no tener fecha el dataframe, no genera ningun campo relacionado a fecha

-Al no tener otras tablas para relacionar no se puede explotar mas lo datos


In [14]:
#Preparacion de datos

In [15]:
X = df.drop(['Target'], axis=1)
y = df.Target
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25, random_state=42) #

AutoFeat

In [16]:
from autofeat import AutoFeatClassifier
#Con estos parametros me muestra los pasos y le seteo la cantidad de pasos
model = AutoFeatClassifier(verbose=1, feateng_steps=1)
new_df = model.fit_transform(X, y) #Esto tarda!

2024-05-16 10:52:39,948 INFO: [AutoFeat] The 1 step feature engineering process could generate up to 42 features.
2024-05-16 10:52:39,949 INFO: [AutoFeat] With 10000 data points this new feature matrix would use about 0.00 gb of space.
2024-05-16 10:52:39,953 INFO: [feateng] Step 1: transformation of original features


2024-05-16 10:52:42,715 INFO: [feateng] Generated 26 transformed features from 6 original features - done.
2024-05-16 10:52:42,717 INFO: [feateng] Generated altogether 28 new features in 1 steps
2024-05-16 10:52:42,718 INFO: [feateng] Removing correlated features, as well as additions at the highest level
2024-05-16 10:52:42,727 INFO: [feateng] Generated a total of 4 additional features
2024-05-16 10:52:42,772 INFO: [featsel] Feature selection run 1/5


[featsel] Scaling data...done.


2024-05-16 10:53:14,513 INFO: [featsel] Feature selection run 2/5
2024-05-16 10:53:47,541 INFO: [featsel] Feature selection run 3/5
2024-05-16 10:54:20,260 INFO: [featsel] Feature selection run 4/5
2024-05-16 10:54:53,500 INFO: [featsel] Feature selection run 5/5
2024-05-16 10:55:25,249 INFO: [featsel] 5 features after 5 feature selection runs
2024-05-16 10:55:25,255 INFO: [featsel] 5 features after correlation filtering
2024-05-16 10:55:35,394 INFO: [featsel] 5 features after noise filtering
2024-05-16 10:55:35,396 INFO: [AutoFeat] Final dataframe with 6 feature columns (0 new).
2024-05-16 10:55:35,396 INFO: [AutoFeat] Training final classification model.
2024-05-16 10:55:36,295 INFO: [AutoFeat] Trained model: largest coefficients:
2024-05-16 10:55:36,295 INFO: [-58.7679747]
2024-05-16 10:55:36,296 INFO: 0.183915 * Torque [Nm]
2024-05-16 10:55:36,297 INFO: 0.122072 * Air temperature [K]
2024-05-16 10:55:36,297 INFO: 0.010615 * Tool wear [min]
2024-05-16 10:55:36,298 INFO: 0.006944 * R

In [17]:
new_df.head(3)

,UDI,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min]
0,0.0,298.1,308.6,1551.0,42.8,0.0
1,1.0,298.2,308.7,1408.0,46.3,3.0
2,2.0,298.1,308.5,1498.0,49.4,5.0


In [18]:
df_test = model.transform(X_test)
print("Final R^2: %.4f" % model.score(new_df, y))
#y_pred = model.predict(X_test)

Final R^2: 0.8116


In [19]:
X_train = model.transform(X_train)

In [20]:
X_test = model.transform(X_test)

Creacion de variables manuales

In [21]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(7500, 6)
(2500, 6)
(7500,)
(2500,)


In [22]:
###FIN FEATURE

*****MODELS*****

**TPOT**

In [23]:
# Importo las librerias
from tpot import TPOTClassifier

C:\Users\djoglar\AppData\Roaming\Python\Python311\site-packages\tpot\builtins\__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


Defino como metrica el Recall ya que me interesa que detecte mas a los casos positivos, aunque de falsos positivos

In [24]:
generations=200
population_size=10
pipeline_optimizer = TPOTClassifier(
    generations=generations,
    population_size=population_size,
    verbosity=2,
    random_state=42,
    memory='auto',
    n_jobs=-1,
    scoring='recall') #Me interesa que ataque a los positivos

In [25]:
%%time
pipeline_optimizer.fit(X_train, y_train)

Optimization Progress:   0%|          | 0/2010 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.5622641509433962

Generation 2 - Current best internal CV score: 0.5622641509433962

Generation 3 - Current best internal CV score: 0.5622641509433962

Generation 4 - Current best internal CV score: 0.5622641509433962

Generation 5 - Current best internal CV score: 0.6296995108315863

Generation 6 - Current best internal CV score: 0.6296995108315863

Generation 7 - Current best internal CV score: 0.6296995108315863

Generation 8 - Current best internal CV score: 0.6296995108315863

Generation 9 - Current best internal CV score: 0.6296995108315863

Generation 10 - Current best internal CV score: 0.6296995108315863

Generation 11 - Current best internal CV score: 0.6296995108315863

Generation 12 - Current best internal CV score: 0.6296995108315863

Generation 13 - Current best internal CV score: 0.6296995108315863

Generation 14 - Current best internal CV score: 0.6479385045422781

Generation 15 - Current best internal CV score: 0.647938

TPOTClassifier(generations=200, memory='auto', n_jobs=-1, population_size=10,
               random_state=42, scoring='recall', verbosity=2)

In [26]:
pipeline_optimizer.export('tpot_export_default_pipeline.py')

In [27]:
y_pred_tpot = pipeline_optimizer.predict(X_test)

C:\Users\djoglar\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\djoglar\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [28]:
df_resultados_tpot = pd.DataFrame({'Real': y_test, 'Predicho': y_pred_tpot})

In [29]:
# Aplicar la función a cada fila del DataFrame para crear la nueva columna
df_resultados_tpot['Puntaje'] = df_resultados_tpot.apply(lambda row: asignar_puntajes(row['Real'], row['Predicho']), axis=1)

# Imprimir el DataFrame con la nueva columna
print(df_resultados_tpot)

      Real  Predicho  Puntaje
6252     0         0        0
4684     1         0 -9000000
1731     0         0        0
4742     0         0        0
4521     0         0        0
...    ...       ...      ...
4862     0         0        0
7025     0         0        0
7647     0         0        0
7161     0         0        0
73       0         0        0

[2500 rows x 3 columns]


In [30]:
GANANCIA_FINAL_tpot = df_resultados_tpot['Puntaje'].sum()
GANANCIA_FINAL_tpot

86000000

In [31]:
print("Recall en el conjunto de prueba:", pipeline_optimizer.score(X_test, y_test))

C:\Users\djoglar\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\djoglar\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Recall en el conjunto de prueba: 0.6111111111111112


In [32]:
recall_tpot=pipeline_optimizer.score(X_test, y_test)

C:\Users\djoglar\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\djoglar\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


**H20**

In [33]:
import h2o
from h2o.automl import H2OAutoML

In [34]:
# Convierto los dataframe de pandas a dataframe de H20
#msk = np.random.rand(len(new_df)) < 0.25
#df_train_H2O = new_df[msk]
#df_test_H2O = new_df[~msk]

In [35]:
# Convierto los dataframe de pandas a dataframe de H2O
#train_H2O = h2o.H2OFrame(df_train_H2O)
#test_H2O = h2o.H2OFrame(df_test_H2O)

In [36]:
# Defino cuales columnas son las variables de entrenamiento y cual es el target.
#x_H2O = train_H2O.columns
#y_H2O = "Target"
#x_H2O.remove(y_H2O)
#print(x_H2O)

In [37]:
#x_H2O

In [38]:
# For binary classification, response should be a factor
#train_H2O[y_H2O] = train_H2O[y_H2O].asfactor()
#test_H2O[y_H2O] = test_H2O[y_H2O].asfactor()

In [39]:
# Entreno el modelo
#aml = H2OAutoML(max_models=20, seed=1, max_runtime_secs=60*2, sort_metric = "AUCPR")
#aml.train(x=x_H2O, y=y_H2O, training_frame=train_H2O)

In [40]:
#Convierto la serie a dataframe
y_test_2 = pd.DataFrame(y_test)
y_train_2 = pd.DataFrame(y_train)

# Inicia H2O
h2o.init()

# Convierte tus datos de entrenamiento a objetos H2OFrame
x_train_h2o = h2o.H2OFrame(X_train)
y_train_h2o = h2o.H2OFrame(y_train_2)

# Combina los datos de entrada con la variable objetivo
train_h2o = x_train_h2o.cbind(y_train_h2o)

y_H2O = "Target"

train_h2o[y_H2O] = train_h2o[y_H2O].asfactor()

# Define y configura el objeto H2OAutoML
aml = H2OAutoML(max_models=20, seed=1, max_runtime_secs=60*2, sort_metric="AUCPR")

# Entrena el modelo
aml.train(x=x_train_h2o.names, y=y_train_h2o.names[0], training_frame=train_h2o)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 21+35-LTS-2513, mixed mode, sharing)
  Starting server from C:\Users\djoglar\AppData\Roaming\Python\Python311\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\djoglar\AppData\Local\Temp\tmp8inds5mg
  JVM stdout: C:\Users\djoglar\AppData\Local\Temp\tmp8inds5mg\h2o_djoglar_started_from_python.out
  JVM stderr: C:\Users\djoglar\AppData\Local\Temp\tmp8inds5mg\h2o_djoglar_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,America/Argentina/Buenos_Aires
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,2 months and 2 days
H2O_cluster_name:,H2O_from_python_djoglar_5qcosu
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,5.910 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
11:36:56.182: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_grid_1_AutoML_1_20240516_113656_model_5


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    41                 41                          70291                  14           17           16.9024       58            203           131.073

ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.0019228948129370034
RMSE: 0.043850824541130394
LogLoss: 0.011078545441689876
Mean Per-Class Error: 0.00027651043826904464
AUC: 0.9999813588468583
AUCPR: 0.9994780837631365
Gini: 0.9999627176937167

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4716480846903713
       0     1    Error    Rate
-----  ----  ---  -------  ------------
0      7229  4    0.0006   (4.0/7233.0)
1      0     267  0        (0.0/267.0)
Total  7229  271  0.0005   (4.0/7500.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.471648     0.992565  173
max f2                       0.471648     0.997013  173
max f0point5                 0.520298     0.992509  169
max accuracy                 0.520298     0.999467  169
max precision                0.996342     1         0
max recall                   0.471648     1         173
max specificity              0.996342     1         0
max absolute_mcc             0.471648     0.992318  173
max min_per_class_accuracy   0.471648     0.999447  173
max mean_per_class_accuracy  0.471648     0.999723  173
max tns                      0.996342     7233      0
max fns                      0.996342     266       0
max fps                      0.000606869  7233      399
max tps                      0.471648     267       173
max tnr                      0.996342     1         0
max fnr                      0.996342     0.996255  0
max fpr                      0.000606869  1         399
max tpr                      0.471648     1         173

Gains/Lift Table: Avg response rate:  3,56 %, avg score:  3,57 %
group    cumulative_data_fraction    lower_threshold    lift     cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain     cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  -------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  -------  -----------------  --------------------
1        0.01                        0.966044           28.0899  28.0899            1                0.980959     1                           0.980959            0.280899        0.280899                   2708.99  2708.99            0.280899
2        0.02                        0.895948           28.0899  28.0899            1                0.936919     1                           0.958939            0.280899        0.561798                   2708.99  2708.99            0.561798
3        0.03                        0.704556           28.0899  28.0899            1                0.815525     1                           0.911134            0.280899        0.842697                   2708.99  2708.99            0.842697
4        0.04                        0.182247           15.7303  25                 0.56             0.468693     0.89                        0.800524            0.157303        1                          1473.03  2400               0.995438
5        0.05                        0.0691231          0        20                 0                0.10511      0.712                      

In [41]:
# Ranking de los modelos entrenados
lb = aml.leaderboard
lb.head()

model_id,aucpr,auc,logloss,mean_per_class_error,rmse,mse
GBM_grid_1_AutoML_1_20240516_113656_model_5,0.807035,0.975883,0.053365,0.140091,0.114508,0.0131121
GBM_grid_1_AutoML_1_20240516_113656_model_2,0.795364,0.97313,0.0529245,0.131765,0.116797,0.0136416
GBM_3_AutoML_1_20240516_113656,0.794183,0.970288,0.0558227,0.149316,0.118138,0.0139566
XRT_1_AutoML_1_20240516_113656,0.788249,0.959814,0.101463,0.156185,0.118959,0.0141513
GBM_2_AutoML_1_20240516_113656,0.787163,0.971755,0.0549561,0.13891,0.117405,0.0137838
DRF_1_AutoML_1_20240516_113656,0.785647,0.963084,0.0968978,0.151535,0.119696,0.0143271
GBM_4_AutoML_1_20240516_113656,0.775894,0.968181,0.058948,0.153684,0.122061,0.0148988
GBM_grid_1_AutoML_1_20240516_113656_model_3,0.747247,0.964456,0.0609348,0.156248,0.124428,0.0154823
GBM_5_AutoML_1_20240516_113656,0.691959,0.970404,0.0686311,0.161589,0.127322,0.0162108
GBM_1_AutoML_1_20240516_113656,0.680756,0.963274,0.0674726,0.196466,0.135546,0.0183726


In [42]:
# Descripcion de todos los modelos entrenados
#exa = aml.explain(test_H2O)

In [43]:
best_model = aml.leader

In [44]:
# Convierte tus datos de entrenamiento a objetos H2OFrame
x_test_h2o = h2o.H2OFrame(X_test)
y_test_h2o = h2o.H2OFrame(y_test_2)

# Combina los datos de entrada con la variable objetivo
test_h2o = x_test_h2o.cbind(y_test_h2o)

y_H2O = "Target"

test_h2o[y_H2O] = test_h2o[y_H2O].asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [45]:
top_models = aml.leaderboard.as_data_frame().head(3)['model_id'].values

C:\Users\djoglar\AppData\Roaming\Python\Python311\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


In [46]:
# Crear una lista para almacenar los DataFrames de cada modelo
results_dfs = []

# Iterar sobre los modelos
for model_id in top_models:
    model = h2o.get_model(model_id)
    predictions = model.predict(test_h2o)
    y_pred = predictions[0].as_data_frame().values.flatten()
    
    # Obtener los valores reales de y_test
    y_real = y_test.values
    
    # Aplicar la función de ganancia a las predicciones de cada modelo
    puntajes = [asignar_puntajes(y_real[i], y_pred[i]) for i in range(len(y_real))]
    
    # Crear un DataFrame para el modelo actual
    df = pd.DataFrame({"Valor_Predicho": y_pred, "Valor_Real": y_real, "Puntaje": puntajes})
    
    # Agregar el nombre del modelo como una columna en el DataFrame
    df['Modelo'] = model_id
    
    results_dfs.append(df)

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


C:\Users\djoglar\AppData\Roaming\Python\Python311\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


C:\Users\djoglar\AppData\Roaming\Python\Python311\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


C:\Users\djoglar\AppData\Roaming\Python\Python311\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


In [47]:
#results_df['Predicho'] = results_df.apply(lambda row: 1 if row['Predicciones'] > 0.80 else 0, axis=1)
#results_dfs['Puntaje'] = results_dfs.apply(lambda row: asignar_puntajes(row['Valor_Real'], row['Valor_Predicho']), axis=1)

results_dfs = pd.concat(results_dfs, ignore_index=True)
results_dfs

,Valor_Predicho,Valor_Real,Puntaje,Modelo
0,0,0,0,GBM_grid_1_AutoML_1_20240516_113656_model_5
1,0,1,-9000000,GBM_grid_1_AutoML_1_20240516_113656_model_5
2,0,0,0,GBM_grid_1_AutoML_1_20240516_113656_model_5
3,0,0,0,GBM_grid_1_AutoML_1_20240516_113656_model_5
4,0,0,0,GBM_grid_1_AutoML_1_20240516_113656_model_5
...,...,...,...,...
7495,0,0,0,GBM_3_AutoML_1_20240516_113656
7496,0,0,0,GBM_3_AutoML_1_20240516_113656
7497,0,0,0,GBM_3_AutoML_1_20240516_113656
7498,0,0,0,GBM_3_AutoML_1_20240516_113656


In [48]:
GANANCIAS_H2O = results_dfs.groupby('Modelo')['Puntaje'].sum()

In [49]:
GANANCIAS_H2O = pd.DataFrame(GANANCIAS_H2O)
GANANCIAS_H2O.reset_index(inplace=True)

In [50]:
GANANCIAS_H2O.rename(columns={'Puntaje': 'GANANCIA'}, inplace=True)
GANANCIAS_H2O

,Modelo,GANANCIA
0,GBM_3_AutoML_1_20240516_113656,184000000
1,GBM_grid_1_AutoML_1_20240516_113656_model_2,237000000
2,GBM_grid_1_AutoML_1_20240516_113656_model_5,137000000


In [51]:
#######################################################

##Probar CatBoost Classifier!!
https://www.kaggle.com/code/prashant111/catboost-classifier-in-python/notebook

In [52]:
from catboost import CatBoostClassifier
from sklearn.metrics import recall_score

# Dividir los datos en conjunto de entrenamiento y prueba

# Crear un clasificador CatBoost
cat_model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6, loss_function='Logloss', custom_metric = ['Recall'], random_seed=42)

# Entrenar el modelo
cat_model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=500, verbose=100)

# Hacer predicciones en el conjunto de prueba
y_pred_cat = cat_model.predict(X_test)

recall_cat = recall_score(y_test, y_pred_cat)
print("Recall del modelo:", recall_cat)

0:	learn: 0.4964203	test: 0.4959323	best: 0.4959323 (0)	total: 158ms	remaining: 2m 37s
100:	learn: 0.0322104	test: 0.0462388	best: 0.0462388 (100)	total: 1.43s	remaining: 12.7s
200:	learn: 0.0210766	test: 0.0447733	best: 0.0447733 (200)	total: 2.51s	remaining: 9.99s
300:	learn: 0.0144012	test: 0.0444894	best: 0.0441994 (286)	total: 3.48s	remaining: 8.09s
400:	learn: 0.0102218	test: 0.0461192	best: 0.0441994 (286)	total: 4.41s	remaining: 6.58s
500:	learn: 0.0074131	test: 0.0468098	best: 0.0441994 (286)	total: 5.51s	remaining: 5.49s
600:	learn: 0.0058717	test: 0.0475180	best: 0.0441994 (286)	total: 6.49s	remaining: 4.31s
700:	learn: 0.0045922	test: 0.0483237	best: 0.0441994 (286)	total: 7.56s	remaining: 3.22s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.0441993859
bestIteration = 286

Shrink model to first 287 iterations.
Recall del modelo: 0.6805555555555556


In [53]:
# Crear un DataFrame con las predicciones del modelo y los valores reales
results_cat = pd.DataFrame({'Prediccion': y_pred_cat, 'Valor_Real': y_test})

# Aplicar la función de ganancia a las predicciones y calcular el puntaje final
results_cat['Puntaje'] = results_cat.apply(lambda row: asignar_puntajes(row['Valor_Real'], row['Prediccion']), axis=1)

# Mostrar el DataFrame resultante
GANANCIAS_cat = results_cat['Puntaje'].sum()
GANANCIAS_cat

174000000

**METRICAS FINALES**

*RECALL*

In [54]:
#METRICAS H2O
recall_results=[]
for model_id in top_models:
    model = h2o.get_model(model_id)
    predictions = model.predict(test_h2o)
    y_pred = predictions[0].as_data_frame().values.flatten()
    y_real = y_test.values
    recall = recall_score(y_real, y_pred)
    recall_results.append({'Modelo': model_id, 'Recall': recall})

recall_df_final = pd.DataFrame(recall_results)

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |

C:\Users\djoglar\AppData\Roaming\Python\Python311\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


███████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |

C:\Users\djoglar\AppData\Roaming\Python\Python311\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


███████████████████████████████████████████████████████| (done) 100%


C:\Users\djoglar\AppData\Roaming\Python\Python311\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


In [55]:
others_score =pd.DataFrame ({'Modelo': ['Modelo Tpot', 'Modelo CAT'], 'Recall': [recall_tpot, recall_cat]})  
recall_df_final = pd.concat([recall_df_final,others_score])

In [56]:
recall_df_final

,Modelo,Recall
0,GBM_grid_1_AutoML_1_20240516_113656_model_5,0.652778
1,GBM_grid_1_AutoML_1_20240516_113656_model_2,0.736111
2,GBM_3_AutoML_1_20240516_113656,0.694444
0,Modelo Tpot,0.611111
1,Modelo CAT,0.680556


*AHORRO POR MODELO*

In [57]:
# Agregar una nueva fila al DataFrame
ahorro_models =pd.DataFrame ({'Modelo': ['Sin prediccion', 'Modelo Tpot', 'Modelo CAT'], 'GANANCIA': [y_test_2['Target'].sum()*3000*3000*-1, GANANCIA_FINAL_tpot, GANANCIAS_cat]})  # Agrega los valores que desees para la nueva fila
Ahorro_modelo_final= pd.concat([ahorro_models,GANANCIAS_H2O])

In [58]:
Ahorro_modelo_final

,Modelo,GANANCIA
0,Sin prediccion,-648000000
1,Modelo Tpot,86000000
2,Modelo CAT,174000000
0,GBM_3_AutoML_1_20240516_113656,184000000
1,GBM_grid_1_AutoML_1_20240516_113656_model_2,237000000
2,GBM_grid_1_AutoML_1_20240516_113656_model_5,137000000
